# Start EC2 instance:

In [1]:
#Basic import boto3 libraries
import boto3
import sys
import time

#provided Access Credentialsaccess_key
access_key = '######'
secret_key = "######"

#Using a non-used ec2 instance to test
instance_id = '######' 

In [2]:
count=0
#Establish a connection to EC2 resource using credentials and region_name
ec2 = boto3.resource('ec2', aws_access_key_id=access_key, aws_secret_access_key=secret_key,region_name='us-east-1')
instance = ec2.Instance(instance_id)
                        
response = instance.start()

print(response)
                        
while(True):
    state = ec2.Instance(instance_id).state
    if state['Name'] == 'running':
        print('It is running')
        break
    else:
        print('starting instance')
        time.sleep(5)
        continue

{'StartingInstances': [{'CurrentState': {'Code': 0, 'Name': 'pending'}, 'InstanceId': 'i-00e6293416c87fdcd', 'PreviousState': {'Code': 80, 'Name': 'stopped'}}], 'ResponseMetadata': {'RequestId': '9b057a22-4173-43c3-9506-a5c73c76fe39', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9b057a22-4173-43c3-9506-a5c73c76fe39', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Tue, 20 Jun 2023 10:26:31 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
starting instance
starting instance
It is running


# Get ip address of the running EC2 instance:

In [3]:
instance_ip = ec2.Instance(instance_id).public_ip_address
print(instance_ip)

50.19.57.242


In [4]:
print(ec2.Instance(instance_id).state)

{'Code': 16, 'Name': 'running'}


# Access to remote host via SSH, activate virtual enviroment and fastapi:

In [ ]:
import boto3
import botocore
import paramiko
import requests

key = paramiko.RSAKey.from_private_key_file('./ssh/reid.pem')
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
# cmd = './ultra/bin/activate; cd /home/ubuntu/inference_api; uvicorn main:app --host 0.0.0.0 --port 8000;'
cmd = 'source myjob.sh;'

# Connect/ssh to an instance
try:
    # Here 'ubuntu' is user name and 'instance_ip' is public IP of EC2
    client.connect(hostname=instance_ip, username="ubuntu", pkey=key)

    # Execute a command(cmd) after connecting/ssh to an instance
    stdin, stdout, stderr = client.exec_command(cmd)
    print(stdout.read())

    r = requests.get('http://' + instance_ip + '/')

    print(r)
    # close the client connection once the job is done
#     if()
    client.close()

except Exception(e):
    print(e)

# Stop the running EC2 instance: 

In [ ]:
response = instance.stop()
                        
print(response)

while(True):
    state = ec2.Instance(instance_id).state
    if state['Name'] == 'stopped':
        print('It is stopped')
        break
    else:
        print('stopping instance')
        time.sleep(5)
        continue    

# Utility code for download file from and upload file to AWS S3

In [48]:
#Basic import boto3 libraries
import boto3
import sys
import time

#provided Access Credentialsaccess_key
access_key = "######"
secret_key = "######"

session = boto3.Session(
    aws_access_key_id = access_key,
    aws_secret_access_key = secret_key,
)

s3 = session.resource('s3')

BUCKET_NAME = 'yolo-inference-poc'
BUCKET_FILE_NAME = 'downloads/18.mp4'
# LOCAL_FILE_NAME = './s3_download/' + BUCKET_FILE_NAME
def download_s3_file(s3, bucket_name, bucket_file_name):
    local_file_name = './s3_download/' + BUCKET_FILE_NAME.split('/')[-1]

    my_bucket = s3.Bucket(bucket_name)
    my_bucket.download_file(BUCKET_FILE_NAME, local_file_name)
    return local_file_name

test_output = download_s3_file(s3, BUCKET_NAME, BUCKET_FILE_NAME)

In [49]:
from pathlib import Path

print(Path(test_output).is_file())

True


In [ ]:
import boto3

s3 = boto3.resource('s3')
s3.meta.client.upload_file( test_output, "yolo-inference-poc", "uploads/18.mp4")